In [ ]:
%load_ext autoreload
%autoreload 2

## Exploring the wiki dump

In [ ]:
import wiki

path_base = '/Users/harangju/Developer/data/wiki/'
name_xml = 'enwiki-20190801-pages-articles-multistream.xml.bz2'
name_index = 'enwiki-20190801-pages-articles-multistream-index.txt.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = wiki.Dump(path_xml, path_index)

In [ ]:
%time dump.load_page('Portal:Physics/Topics')
dump.links[:5]

In [ ]:
dump.load_page('Danielle Bassett')
dump.links[:5]

In [ ]:
dump.load_page('Matter', filter_top=True).strip_code()[:200]

## Get index of physics articles

* [all indices on Wikipedia](https://en.wikipedia.org/wiki/Portal:Contents/Indices)
* topics not searched
* international trade ("topics"), theory of constraints (small)
* too big: mathematics, neuroscience

In [ ]:
# natural & physical sciences
topics = ['anatomy', 'biochemistry', 'cognitive science', 'evolutionary biology',
          'genetics', 'immunology', 'molecular biology']
topics += ['chemistry', 'biophysics', 'energy', 'optics', 
           'earth science', 'geology', 'meteorology']
# philosophy
# topics += []
topics += ['philosophy of language', 'philosophy of law', 
           'philosophy of mind', 'philosophy of science']
# social sciences
topics += ['economics', 'accounting', 'education', 'linguistics', 'law', 'psychology', 'sociology']
# technology & applied sciences
topics += ['electronics', 'software engineering', 'robotics']

In [ ]:
links = {}
for topic in topics:
    dump.load_page('Index of %s articles' % topic)
    links[topic] = [str(l) for l in dump.article_links]
    print('Topic "' + topic + '" has ' + str(len(links[topic])) + ' articles.')

In [ ]:
import string

links['physics'] = []
for letter in ['!$@', '0–9'] + list(string.ascii_uppercase):
    dump.load_page('Index of physics articles (%s)' % letter)
    links['physics'].extend([str(l) for l in dump.article_links])
print('Topic "' + 'physics' + '" has ' + str(len(links['physics'])) + ' articles.')

## Build graphs of topics

### 1 network per topic

In [ ]:
import networkx as nx

path_to_save = '/Users/harangju/Box Sync/Research/my papers/wikipedia paper/data/graphs/undated/'

graphs = {}
page_noload = {}
depth = 1
for topic in links.keys():
    print('Graph topic: ' + topic)
    graphs[topic] = nx.DiGraph()
    page_noload[topic] = wiki.Crawler.bfs(graphs[topic], dump, links[topic],
                                          depth_goal = depth, nodes = links[topic])
    nx.write_gexf(graphs[topic], path_to_save + topic + '.gexf')

In [ ]:
for topic, graph in graphs.items():
    graph.remove_nodes_from(nx.isolates(graph))
    nx.write_gexf(graph, path_to_save + topic + '.gexf')

Gephi notes
* node size, fruchterman reingold = [10, 40], force atlas 2 = [4 16]
* text size = [1 1.4]
* preview font size = 5

### 1 network for all topics
i.e., connect all networks into 1 big network

In [ ]:
# big_list = [item for sublist in list(links.values()) for item in sublist]
# big_graph = nx.DiGraph()
# wiki.Crawler.bfs(big_graph, dump, big_list, depth_goal = 2, nodes = big_list)
# nx.write_gexf(big_graph, path_base + 'graphs/big_graph.gexf')